In [1]:
import os
import pyspark

# Start Spark session
conf = pyspark.SparkConf()
conf.setMaster("local").setAppName("My app")

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

In [2]:
from pyspark.sql.functions import col, sum, round, when

In [3]:
# Load fishing subsidies dataset
subsidies = spark \
                .read \
                .option("inferSchema", "true") \
                .option("header", "true") \
                .csv("data/fishing_subsidies.csv")

In [4]:
subsidies.head(5)

[Row(COUNTRY0='AUS', Country1='Australia', VARIABLE2='IFINP', Variable3='I.A. Transfers based on input use', MEASURE4='USD', Measure5='US dollar', YEAR6=2014, Year7=2014, Unit Code='USD', Unit='US Dollar', PowerCode Code=0, PowerCode='Units', Reference Period Code=None, Reference Period=None, Value=1103477.1538706, Flag Codes=None, Flags=None),
 Row(COUNTRY0='AUS', Country1='Australia', VARIABLE2='IFINP', Variable3='I.A. Transfers based on input use', MEASURE4='USD', Measure5='US dollar', YEAR6=2015, Year7=2015, Unit Code='USD', Unit='US Dollar', PowerCode Code=0, PowerCode='Units', Reference Period Code=None, Reference Period=None, Value=2045669.3008909, Flag Codes=None, Flags=None),
 Row(COUNTRY0='AUS', Country1='Australia', VARIABLE2='IFINP', Variable3='I.A. Transfers based on input use', MEASURE4='USD', Measure5='US dollar', YEAR6=2016, Year7=2016, Unit Code='USD', Unit='US Dollar', PowerCode Code=0, PowerCode='Units', Reference Period Code=None, Reference Period=None, Value=145465

In [5]:
subsidies.printSchema()

root
 |-- COUNTRY0: string (nullable = true)
 |-- Country1: string (nullable = true)
 |-- VARIABLE2: string (nullable = true)
 |-- Variable3: string (nullable = true)
 |-- MEASURE4: string (nullable = true)
 |-- Measure5: string (nullable = true)
 |-- YEAR6: integer (nullable = true)
 |-- Year7: integer (nullable = true)
 |-- Unit Code: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- PowerCode Code: integer (nullable = true)
 |-- PowerCode: string (nullable = true)
 |-- Reference Period Code: string (nullable = true)
 |-- Reference Period: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- Flag Codes: string (nullable = true)
 |-- Flags: string (nullable = true)



In [6]:
# Drop unnecessary columns
subsidies = subsidies.drop("Reference Period Code", \
                           "Reference Period", \
                           "Flag Codes", \
                           "Flags", \
                           "PowerCode Code", \
                           "PowerCode", \
                           "YEAR6", \
                           "MEASURE4", \
                           "Measure5")

In [7]:
# Rename columns
subsidies = subsidies.withColumnRenamed("COUNTRY0", "Country Code") \
                .withColumnRenamed("Country1", "Country") \
                .withColumnRenamed("VARIABLE2", "Variable Code") \
                .withColumnRenamed("Variable3", "Variable") \
                .withColumnRenamed("Year7", "Year")

In [8]:
# Check schema
subsidies.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Variable Code: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Unit Code: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: double (nullable = true)



In [9]:
# Retrieve different variables and its corresponding codes
variables = subsidies.select("Variable Code", "Variable").distinct().sort("Variable")
variables.show(n=50, truncate=False)

+-------------+--------------------------------------------------------------------------------------+
|Variable Code|Variable                                                                              |
+-------------+--------------------------------------------------------------------------------------+
|IFINP        |I.A. Transfers based on input use                                                     |
|IFINP_VAR    |I.A.1. Transfers based on variable input use                                          |
|IFINP_FIX    |I.A.2. Transfers based on fixed capital formation                                     |
|IFINP_FIX_V  |I.A.2.1.Support to vessel construction/purchase                                       |
|IFINP_FIX_M  |I.A.2.2.Support to modernisation                                                      |
|IFINP_FIX_O  |I.A.2.3.Support to other fixed costs                                                  |
|IFINC        |I.B. Transfers based on fishers income                    

In [10]:
# Get list of all variable codes
var_codes = list(subsidies.select("Variable Code").distinct().toPandas()["Variable Code"])

In [11]:
# Divide up variable codes by which ones correspond to government support and which ones to industry payments to the government
payments = ['FCRRE', 'FCRIN', 'FCRMG', 'FCROT']
support = [code for code in var_codes if code not in payments]

In [12]:
# Aggregate sum of government support per country/year
subsidies_support = subsidies.filter(col("Variable Code").isin(support)).groupBy("Country Code", "Country", "Year", "Unit Code").agg(round(sum("Value"), 2).alias("Total Support"))

In [13]:
# Aggregate sum of industry payments per country/year
subsidies_payments = subsidies.filter(col("Variable Code").isin(payments)).groupBy("Country Code", "Country", "Year", "Unit Code").agg(round(sum("Value"), 2).alias("Total Payments"))

In [14]:
# Combine stats into one dataframe
subsidies_stats = subsidies_support.alias("df1").join( \
                                                      subsidies_payments.alias("df2"), \
                                                      (col("df1.Country Code") == col("df2.Country Code")) & \
                                                      (col("df1.Country") == col("df2.Country")) & \
                                                      (col("df1.Year") == col("df2.Year")) & \
                                                      (col("df1.Unit Code") == col("df2.Unit Code")) \
                                                     ).select( \
                                                              col("df1.Country Code").alias("Country Code"), \
                                                              col("df1.Country").alias("Country"), \
                                                              col("df1.Year").alias("Year"), \
                                                              col("df1.Unit Code").alias("Unit Code"), \
                                                              col("df1.Total Support").alias("Total Support"), \
                                                              col("df2.Total Payments").alias("Total Payments") \
                                                             ).orderBy("Country Code", "Year")

In [15]:
# Load the transformed data into MongoDB
import json
import pymongo
from dotenv import dotenv_values

In [16]:
# Retrieve MongoDB creds
config = dotenv_values("creds.env")

user = config['user']
password = config['password']

# Connect to MongoDB
uri = "mongodb+srv://" + user + ":" + password + "@cluster0.6jfc5iw.mongodb.net/"
client = pymongo.MongoClient(uri)
db = client["gfw"]
collection = db["subsidies"]

In [29]:
subsidies_dict = subsidies_stats.toPandas().to_dict("records")

In [31]:
collection.insert_many(subsidies_dict)

In [32]:
client.close()